Source: 
- Deploy Mistral 7B Instruct in Sagemaker as an endpoint
- https://medium.com/@furkan_sevimli/deploy-mistral-7b-instruct-v0-1-on-amazon-sagemaker-daa5a6c48f1b

Steps to do: 
- Make sure you have been given access to the Mistral 7B 
Instruct v2.0 model in HuggingFace

In [1]:
!pip install "sagemaker>=2.199.0" --upgrade --quiet

In [2]:
!pip install -U "huggingface_hub[cli]"

In [15]:
!huggingface-cli login --token "<YOUR_HF_TOKEN>"

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ec2-user/.cache/huggingface/token
Login successful


In [13]:
import sagemaker
import boto3
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [5]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [19]:
# Hub Model configuration. https://huggingface.co/models
hub = {
 'HF_MODEL_ID':'mistralai/Mistral-7B-Instruct-v0.1',
 'SM_NUM_GPUS': json.dumps(1),
'HUGGING_FACE_HUB_TOKEN': "<YOUR_HF_TOKEN>"
}

In [20]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
 image_uri=get_huggingface_llm_image_uri("huggingface",version="1.1.0"),
 env=hub,
 role=role, 
)

In [21]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
 initial_instance_count=1,
 instance_type="ml.g5.4xlarge",
 container_startup_health_check_timeout=300,
  )

--------!

In [22]:
# Prompt to generate
prompt=f"<s> [INST] Can you suggest 5 science fiction movies? [/INST] "

# Generation arguments
payload = {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.9,
    "top_k": 50,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "return_full_text": False,
    "stop": ["</s>"]
}

In [23]:
chat = predictor.predict({"inputs":prompt, "parameters":payload})

print(chat[0]["generated_text"])

1. Blade Runner (1982) - directed by Ridley Scott, starring Harrison Ford.
2. The Matrix (1999) - directed by the Wachowskis, starring Keanu Reeves.
3. E.T. the Extra-Terrestrial (1982) - directed by Steven Spielberg, starring Henry Thomas.
4. 2001: A Space Odyssey (1968) - directed by Stanley Kubrick, starring Keir Dullea and Gary Lockwood.
5. Inception (2010) - directed by Christopher Nolan, starring Leonardo DiCaprio, Joseph Gordon-Levitt, and Ellen Page.
